In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType
from datetime import date

spark = SparkSession.builder.appName("Practice").getOrCreate()

# Define Schema
schema = StructType([
    StructField("emp_id", IntegerType(), True),
    StructField("emp_name", StringType(), True),
    StructField("dept", StringType(), True),
    StructField("salary", IntegerType(), True),
    StructField("join_date", DateType(), True)
])

# Sample Data
data = [
    (1, "Amit", "IT", 70000, date(2022,1,10)),
    (2, "Rahul", "IT", 60000, date(2021,5,12)),
    (3, "Priya", "IT", 60000, date(2023,3,15)),
    (4, "Neha", "HR", 50000, date(2020,7,19)),
    (5, "Karan", "HR", 80000, date(2019,11,1)),
    (6, "Rohit", "Finance", 90000, date(2018,4,23)),
    (7, "Simran", "Finance", 40000, date(2022,8,30)),
    (8, "Arjun", "IT", 75000, date(2020,9,14))
]

employee = spark.createDataFrame(data, schema)

employee.show()
employee.printSchema()

26/02/21 00:12:22 WARN Utils: Your hostname, ASHUTOSHs-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.1.100 instead (on interface en0)
26/02/21 00:12:22 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/02/21 00:12:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


+------+--------+-------+------+----------+
|emp_id|emp_name|   dept|salary| join_date|
+------+--------+-------+------+----------+
|     1|    Amit|     IT| 70000|2022-01-10|
|     2|   Rahul|     IT| 60000|2021-05-12|
|     3|   Priya|     IT| 60000|2023-03-15|
|     4|    Neha|     HR| 50000|2020-07-19|
|     5|   Karan|     HR| 80000|2019-11-01|
|     6|   Rohit|Finance| 90000|2018-04-23|
|     7|  Simran|Finance| 40000|2022-08-30|
|     8|   Arjun|     IT| 75000|2020-09-14|
+------+--------+-------+------+----------+

root
 |-- emp_id: integer (nullable = true)
 |-- emp_name: string (nullable = true)
 |-- dept: string (nullable = true)
 |-- salary: integer (nullable = true)
 |-- join_date: date (nullable = true)



In [3]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

order_schema = StructType([
    StructField("order_id", IntegerType(), True),
    StructField("customer_id", IntegerType(), True),
    StructField("product", StringType(), True),
    StructField("amount", IntegerType(), True)
])

order_data = [
    (101, 1, "Laptop", 50000),
    (102, 2, "Mobile", 20000),
    (103, 1, "Keyboard", 2000),
    (104, 3, "Laptop", 50000),
    (105, 2, "Mouse", 1000),
    (106, 3, "Monitor", 10000)
]

orders = spark.createDataFrame(order_data, ["order_id", "emp_id", "product", "amount"])

orders.show()

+--------+------+--------+------+
|order_id|emp_id| product|amount|
+--------+------+--------+------+
|     101|     1|  Laptop| 50000|
|     102|     2|  Mobile| 20000|
|     103|     1|Keyboard|  2000|
|     104|     3|  Laptop| 50000|
|     105|     2|   Mouse|  1000|
|     106|     3| Monitor| 10000|
+--------+------+--------+------+



In [4]:
employee.show(),orders.show()

+------+--------+-------+------+----------+
|emp_id|emp_name|   dept|salary| join_date|
+------+--------+-------+------+----------+
|     1|    Amit|     IT| 70000|2022-01-10|
|     2|   Rahul|     IT| 60000|2021-05-12|
|     3|   Priya|     IT| 60000|2023-03-15|
|     4|    Neha|     HR| 50000|2020-07-19|
|     5|   Karan|     HR| 80000|2019-11-01|
|     6|   Rohit|Finance| 90000|2018-04-23|
|     7|  Simran|Finance| 40000|2022-08-30|
|     8|   Arjun|     IT| 75000|2020-09-14|
+------+--------+-------+------+----------+

+--------+------+--------+------+
|order_id|emp_id| product|amount|
+--------+------+--------+------+
|     101|     1|  Laptop| 50000|
|     102|     2|  Mobile| 20000|
|     103|     1|Keyboard|  2000|
|     104|     3|  Laptop| 50000|
|     105|     2|   Mouse|  1000|
|     106|     3| Monitor| 10000|
+--------+------+--------+------+



(None, None)

In [30]:
from pyspark.sql.functions import sum
from pyspark.sql.window import Window

window_range = Window.partitionBy("dept").orderBy("salary")

window_rows = Window.partitionBy("dept") \
                    .orderBy("salary") \
                    .rowsBetween(Window.unboundedPreceding, Window.currentRow)

employee.withColumn("range_sum", sum("salary").over(window_range)) \
        .withColumn("rows_sum", sum("salary").over(window_rows)) \
        .show()

+------+--------+-------+------+----------+---------+--------+
|emp_id|emp_name|   dept|salary| join_date|range_sum|rows_sum|
+------+--------+-------+------+----------+---------+--------+
|     7|  Simran|Finance| 40000|2022-08-30|    40000|   40000|
|     6|   Rohit|Finance| 90000|2018-04-23|   130000|  130000|
|     4|    Neha|     HR| 50000|2020-07-19|    50000|   50000|
|     5|   Karan|     HR| 80000|2019-11-01|   130000|  130000|
|     2|   Rahul|     IT| 60000|2021-05-12|   120000|   60000|
|     3|   Priya|     IT| 60000|2023-03-15|   120000|  120000|
|     1|    Amit|     IT| 70000|2022-01-10|   190000|  190000|
|     8|   Arjun|     IT| 75000|2020-09-14|   265000|  265000|
+------+--------+-------+------+----------+---------+--------+



In [43]:
from pyspark.sql.functions import col,dense_rank,row_number

window = Window.partitionBy('dept').orderBy(col('salary').desc())
highest_sal_per_dept = employee.withColumn('rnk',row_number().over(window)).filter(col('rnk') == 3).select('emp_name','dept','salary')
highest_sal_per_dept.show()

+--------+----+------+
|emp_name|dept|salary|
+--------+----+------+
|   Rahul|  IT| 60000|
+--------+----+------+



In [46]:
from pyspark.sql.functions import max, when

result = employee \
    .withColumn("rn", row_number().over(window)) \
    .groupBy("dept") \
    .agg(
        max(when(col("rn") == 2, col("salary"))).alias("third_salary")
    )

result.show()

+-------+------------+
|   dept|third_salary|
+-------+------------+
|Finance|       40000|
|     HR|       50000|
|     IT|       70000|
+-------+------------+

